In [7]:
function f!(y,x)
    y[1] = 3*x^2
    y[2] = x^4
    nothing
end

y = [0,0]
f!(y,3)
@show y

y = [0,0]
ForwardDiff.derivative(f!, y, 3) #Syntax .derivative(funktion, Variable, Stelle der Ableitung)

y = [27, 81]


2-element Vector{Int64}:
  18
 108

In [10]:
g(x) = 3*x[1]^2+x[2]^4
g([1,2])
ForwardDiff.gradient(g,  [1,2])

h(x,p) = p[1] *x[1]^2 +p[2]*x[2]^4
h([1,2], [3,1])
ForwardDiff.gradient(x -> h(x, [3,1]),[1,2])

2-element Vector{Int64}:
  6
 32

Reverse Differentiation

In [9]:
using ReverseDiff

# Define a simple function
f(x) = x^2 + sin(x)

# Define the input value
x = 2.0

# Compute the gradient
gradient_result = gradient(f, x)

println("Gradient at x = $x: $gradient_result")

Gradient at x = 2.0: (3.5838531634528574,)


In [10]:
using ForwardDiff

# Define a simple function
f(x) = x^2 + sin(x)

# Define the input value
x = 2.0

# Compute the gradient
gradient_result = ForwardDiff.derivative(f, x)

println("Gradient at x = $x: $gradient_result")

Gradient at x = 2.0: 3.5838531634528574


REVERSEDIFF FASTER THAN FORWARDDIFF

In [27]:
using ReverseDiff
using BenchmarkTools

# Define a simple function
f(x) = sum(x .^ 2)

# Input values
x = rand(1000)

# Benchmark ReverseDiff (backward differentiation)
@btime gradient(f, x)

# Benchmark ForwardDiff (forward differentiation)
using ForwardDiff
@btime ForwardDiff.gradient(f, x)

  7.175 μs (18 allocations: 16.28 KiB)


  2.076 ms (173 allocations: 8.44 MiB)


1000-element Vector{Float64}:
 1.4404907492776222
 0.3641136609090274
 0.6578963057858984
 0.860395528110609
 1.2044421778501537
 1.2142689380070693
 0.3290150644767269
 1.278188466856067
 1.7407254585747634
 1.9545773140781482
 ⋮
 0.5680727783008848
 1.706572468315645
 0.66622831666345
 1.19005750345353
 1.0382907953611165
 1.0958346723238006
 0.16621766339843091
 0.5207973292796837
 0.5442212367900865

µs: Microsekunden 1µs = 1*10^-6 s
ms: Millisekunden 1ms = 1*10^-3 s
ReverseDiff: 7*10^-6s = 0,000007s
ForwardDiff: 2,049*10^-3s = 0,002049s

In [39]:
using ReverseDiff
using BenchmarkTools

# Define a simple function
f(x) = sum(x .^ 2)

# Input values
x = [1,2,3,4,5]

# Benchmark ReverseDiff (backward differentiation)
ableitungsrev = gradient(f,x)
@btime ableitungsrev


# Benchmark ForwardDiff (forward differentiation)
using ForwardDiff
ableitungsforw = ForwardDiff.gradient(f, x)
@btime ableitungsforw

  5.300 ns (0 allocations: 0 bytes)


  5.300 ns (0 allocations: 0 bytes)


false

REVERSE MODE VS. FORWARD MODE

Input-Dimension: Vektor
Output-Dimension: Skalar

In [6]:
using BenchmarkTools
f(x::Vector) = sin(x[1]) + prod(x[2:end]);
x = vcat(pi/4, 2:4)
f(x)

using Zygote
@btime reversemode = gradient(f,x)

using ForwardDiff
@btime forwardmode = ForwardDiff.gradient(f, x)

  2.911 μs (25 allocations: 1.94 KiB)


  800.000 ns (5

 allocations: 784 bytes)


4-element Vector{Float64}:
  0.7071067811865476
 12.0
  8.0
  6.0

In diesem Fall ist reverse mode marginal langsamer als forward mode, denn 
Input-Dimension = 2
Output-Dimension = 1
Es lässt sich erst ein deutlich größerer Unterschied feststellen, wenn man eine komplexere Funktion nimmt.

In [33]:
reversemode == forwardmode
# ergibt false, da forwardmode ist ein Vektor und reversemode ist ein Tupel

false

Input-Dimension: Vektor mit 200 Einträgen
Output-Dimension: Skalar

In [8]:
using BenchmarkTools
f(x) = sum(x .^ 2);
x = rand(200);

using Zygote
@btime gradient(f,x);

using ForwardDiff
@btime ForwardDiff.gradient(f, x);

  3.275 μs (25 allocations: 8.66 KiB)


  111.800 μs (39 allocations: 369.45 KiB)


reverse mode schneller als forward mode
Input-Dimension=200 >> Ouput-Dimension=1
reverse mode: 1,68*10^-6s = 0,00000168s
forward mode: 58*10^-6s = 0,000058s

In [26]:
using BenchmarkTools
using Zygote
@time hessian(f,x)

using ForwardDiff
@time ForwardDiff.hessian(f,x);

  0.000515 seconds (428 allocations: 1.680 MiB)
  0.050726 seconds (623 allocations: 75.508 MiB, 28.18% gc time)


Input-Dimension: Vektor
Output-Dimension: Matrix

In [38]:
using BenchmarkTools
f(x) = [[(x[1]^2+sin(x[2])), (exp(x[2])+3), x[1]*x[2]] [(2*x[1]^4*x[2]), exp(x[1]), (x[1]+x[2])] [sin(x[1]), 5x[1]+x[2]^2, 1]]
x = [1 2]
f(x)

using Zygote
print("Die Berechnung der Jacobi-Matrix mittels reverse diff dauert")
@btime jacobian(f,x)

using ForwardDiff
print("Die Berechnung der Jacobi-Matrix mittels forward diff dauert")
@btime ForwardDiff.jacobian(f, x);

Die Berechnung der Jacobi-Matrix mittels reverse diff dauert

  246.000 μs (817 allocations: 39.16 KiB)
Die Berechnung der Jacobi-Matrix mittels forward diff dauert

  1.990 μs (14 allocations: 1.28 KiB)


Input-Dimension: 2x1 Vektor
Ouput-Dimension: 3x3 Matrix
Input-Dimension < Output-Dimension
Berechnung Jacobi-Matrix
reverse mode: 164,4*10^-6s = 0,0001644s
forward mode: 771,698*10^-9s = 0,000000771698s
-> forward mode ist deutlich schnller

AUTODIFF: BACKPROPAGATION

AUTODIFF IN LAYERS
Trainieren des neuronalen Netzes, indem die Gewichte verbessert werden. Das geschieht durch Anwendung des Gradientenverfahrens, denn es muss das Minimum der Funktion E gefunden werden. 

In [ ]:
input = [0,1,2,3,4,5,6,7,8,9]
ouput = [0,1,2,3,4,5,6,7,8,9]
gewichtsmatrix = randn()

In [52]:
using MaNN
# Struktur des neuronalen Netzes
# implementiere jeweils 4 Gewichts- und 4 Biasmatrizen
mutable struct NeuronalNetwork
    W1::Matrix{Float64}
    b1::Vector{Float64}
    W2::Matrix{Float64}
    b2::Vector{Float64}
    W3::Matrix{Float64}
    b3::Vector{Float64}
    W4::Matrix{Float64}
    b4::Vector{Float64}
end

In [53]:
# Funktion, die die Gewichts- und Biasmatrizen in ihrer jeweiligen Größe mit zufälligen Werten initialisiert
# W hat so viele Zeilen wie Neuronen im nächsten hidden layer sind und so viele Spalten wie die Anzahl der Inputs ist
# b ist ein Vektor mit so vielen Zeilen wie Neuronen in dem nächsten hidden layer sind
function initialize_network(input_size, hidden_size1, hidden_size2, output_size)
    W1 = randn(hidden_size1, input_size)
    b1 = zeros(hidden_size1)
    W2 = randn(hidden_size2, hidden_size1)
    b2 = zeros(hidden_size2)
    W3 = randn(output_size, hidden_size2)
    b3 = zeros(output_size)
    W4 = randn(output_size, hidden_size2)
    b4 = zeros(output_size)
    return NeuralNetwork(W1, b1, W2, b2, W3, b3)
end

initialize_network (generic function with 1 method)

In [55]:
#Daten müssen irgendwie noch umgeändert werden, denn das funktioniert so nicht, da trainset nicht einfach nur 
# ein array oder int ist
data = 1

using Zygote

function backpropagate!(model, data, y_true, learning_rate)
    # Forward pass
    # zi ist die Summe der gewichteten Eingangsdaten
    # yi ist der hidden state, also die Aktivierungsfunktion auf die zi angewandt
    z1 = NeuronalNetwork.W1 * data .+ NeuronalNetwork.b1
    y1 = MaNN.leakyrelu(z1)
    z2 = NeuronalNetwork.W2 * y1 .+ NeuronalNetwork.b2
    y2 = MaNN.leakyrelu(z2)
    z3 = NeuronalNetwork.W3 * y2 .+ NeuronalNetwork.b3
    y3 = MaNN.leakyrelu(z3)
    y4 = MaNN.softmax(y3)

    # Compute loss
    # a4 ist der Output des neuronalen Netzes
    loss = MaNN.cross_entropy(y_true, y4)

    # Backpropagation
    ## unbedingt mit zygote(reversediff) schreiben
    ableitungW4 = gradient(loss, W4)
    W4 = NeuronalNetwork.W4 - learning_rate*ableitungW4
    ableitungW3 = gradient(loss, W3)
    W3 = NeuronalNetwork.W3 - learning_rate*ableitungW3
    ableitungW2 = gradient(loss, W2)
    W2 = NeuronalNetwork.W2 - learning_rate*ableitungW2
    ableitungW1 = gradient(loss, W1)
    W1 = NeuronalNetwork.W1 - learning_rate*ableitungW1
    return loss
end

backpropagate! (generic function with 1 method)

In [ ]:
function train_hardcoded(model, data, labels, learning_rate, num_epochs)
    for epoch in 1:num_epochs
        total_loss = 0
        for i in 1:size(trainset, 2)
            x = trainset[:,i]
            y_true = labels[:,i]
            loss = backporpagate!(model, x, y_true, learning_rate)
            total_loss += loss
        end
        println("Epoch:" +epoch+ "Loss:" +total_loss)
    end
end

In [43]:
input_size = 784
hidden_size1 = 256
hidden_size2 = 128
output_size = 10
labels = [i == j ? 1.0:0.0 for i in 1:output_size, j in 1:100]
learning_rate = 0.1
num_epochs = 10
train_hardcoded(model, data, labels, learning_rate, num_epochs)

ErrorException: syntax: space required before colon in "?" expression

In [ ]:
## implementiere jeweils 3 Gewichts- und 3 Biasmatrizen
# Define the neural network architecture
using Zygote

function backpropagate!(model, data, y_true, learning_rate)
    for data in trainset
        data = trainset.features
        ableitung = gradient(model) do m
            result = m(input)
            loss = MaNN.cross_entropy(y_true, result)
            return loss
            print(loss)
        end
        #Gewichte updaten
        Dense.weights -= learning_rate*ableitung[weights]
        Dense.biases -= learning_rate*ableitung[biases]
    end
end

backpropagate!(model, trainset.features)

In [1]:
using Flux
x = [2, 1];
y = [2, 0];

In [5]:
gs = gradient(Flux.params(x, y)) do
    f(x, y)
  end


ErrorException: Output should be scalar; gradients are not defined for output var"#f#18"()

In [ ]:
Grads(...)
gs[x]

gs[y]